In [ ]:
# %run "z_report_data/get required data.ipynb"

In [ ]:
from datetime import datetime
from z_report_data.variables import app_vars
import pandas as pd

In [ ]:
store_name = app_vars.STORE_NAME
transactions = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions.csv',
        parse_dates=['CreateDate', 'DeleteDate'],
        date_format='%Y-%m-%d %H:%M:%S',
        dtype={'Store': 'category'},
        usecols=['Id', 'CreateDate', 'HeadCount', 'PrivilegeHeadCount', 'Gross', 'Tags', 'DeleteDate', 'SyncInfoId']
    )
    .dropna(how='all')
)

after_start = transactions['CreateDate'] >= app_vars.START_DATE
before_end = transactions['CreateDate'] < app_vars.END_DATE

transactions = transactions[after_start & before_end]

In [ ]:
sync_infos = (
    pd
    .read_csv(
        f'./assets/{store_name}_transactions_sync_info.csv',
        usecols=['Id', 'LocalReference', 'ExternalReference'],
        dtype={'ExternalReference': 'int64', 'LocalReference':'int64'}
    )
)

sync_infos.info()

In [ ]:
discounts = (
    pd
    .read_csv(
        f'./assets/{store_name}_transaction_discounts.csv',
        usecols=['Amount', 'TransactionId', 'Type', 'DeleteDate']
    )
    .where(lambda x : x['DeleteDate'].isna())
)

discounts

In [ ]:
merged = (
    transactions
    .merge(
        sync_infos,
        how='inner', left_on='SyncInfoId', right_on='Id',
    )
    .dropna(how='all')
    .drop(['Id_y', 'SyncInfoId', 'LocalReference'], axis=1)
    .rename(columns={'Id_x': 'Id'})
)

merged.head()

In [ ]:
trs = merged
valid = trs[trs['DeleteDate'].isna()]
cancelled = trs[trs['DeleteDate'].isna() == False]

tr_discs = discounts.merge(trs, how='inner', left_on='TransactionId', right_on='Id')
seniors = tr_discs['Type'] == 'SeniorDiscount'
pwds = tr_discs['Type'] == 'Pwd'
priv_discs = tr_discs[seniors | pwds]
opens = tr_discs['Type'] == 'OpenDiscount'
open_discounts = tr_discs[opens]
vat_exes = tr_discs['Type'] == 'VatExempt'
vat_exclusives = tr_discs[vat_exes]

gross = valid['Gross'].sum()
count = valid['Id'].count()
cancelCount = cancelled['Id'].count()
cancelGross = cancelled['Gross'].sum()
priv_disc = priv_discs['Amount'].sum()
open_disc = open_discounts['Amount'].sum()
vat_of_vat_exempt = vat_exclusives['Amount'].sum()
vat_exempt = vat_of_vat_exempt / 0.12
vatable = gross - vat_exempt
vat = vatable * 0.12
trans_start = valid['ExternalReference'].min()
trans_end = valid['ExternalReference'].max()

date_format = '%m/%d/%Y %I:%M %p'
report_date_from = valid['CreateDate'].min()
report_date_to = valid['CreateDate'].max()

with open(f"./output/{store_name}-z-report.txt", 'w', encoding='utf-8') as file:
    file.write(f"{'Consolidated Z Report':^42}\n")
    for col in app_vars.header.split('#'):
        file.write(f"{col:^42}\n")

    file.write("\n")
    file.write(f"{'Print Date:':<21}{datetime.today().strftime(date_format):>21}\n")
    file.write(f"{'Report Date From:':<21}{report_date_from.strftime(date_format):>21}\n")
    file.write(f"{'Report Date To:':<21}{report_date_to.strftime(date_format):>21}\n")
    file.write("=" * 42)
    file.write("\n")
    file.write(f"{'Trans Count:':<21}{count:>21}\n")
    file.write(f"{'Void Count:':<21}{cancelCount:>21}\n")
    file.write(f"{'Void Amount:':<21}{cancelGross:>21,.2f}\n")
    file.write(f"{'Change Fund:':<21}{10000:>21,.2f}\n")
    file.write(f"{'Gross Sales:':<21}{gross:>21,.2f}\n")
    file.write(f"{'Priv. Discount:':<21}{priv_disc:>21,.2f}\n")
    file.write(f"{'Other Discount:':<21}{open_disc:>21,.2f}\n")
    file.write(f"{'VAT Amount:':<21}{vat:>21,.2f}\n")
    file.write(f"{'VATABLE:':<21}{vatable:>21,.2f}\n")
    file.write(f"{'VAT Exempt:':<21}{vat_exempt:>21,.2f}\n")
    file.write(f"{'Zero-Rated:':<21}{0.00:>21,.2f}\n")
    file.write(f"{'Trans Start:':<21}{trans_start:>21}\n")
    file.write(f"{'Trans End:':<21}{trans_end:>21}\n")
    file.write(f"{'Reset Counter:':<21}{0:>21}\n")
    file.write(f"{'Checked By:':<21}{'mark':>21}\n")

with open(f'./outputs/{store_name}-z-report.txt', 'r', encoding='utf-8') as f:
    print(f.read())